<a href="https://colab.research.google.com/github/yaicianis/Face-Mask-Detection/blob/main/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [3]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing Face Mask Dataset


In [4]:
# API to fetch the dataset from Kaggle
!kaggle datasets download -d omkargurav/face-mask-dataset

Dataset URL: https://www.kaggle.com/datasets/omkargurav/face-mask-dataset
License(s): unknown
 97% 158M/163M [00:01<00:00, 144MB/s]
100% 163M/163M [00:01<00:00, 123MB/s]


In [5]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/face-mask-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [6]:
ls

data/  face-mask-dataset.zip  sample_data/


importing the dependencies

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

In [11]:
with_mask_files=os.listdir('/content/data/with_mask')
print(with_mask_files[0:5])
print(with_mask_files[-5:])

['with_mask_3310.jpg', 'with_mask_2294.jpg', 'with_mask_2790.jpg', 'with_mask_1926.jpg', 'with_mask_1152.jpg']
['with_mask_1177.jpg', 'with_mask_1375.jpg', 'with_mask_2639.jpg', 'with_mask_1889.jpg', 'with_mask_2445.jpg']


In [12]:
without_mask_files=os.listdir('/content/data/without_mask')
print(without_mask_files[0:5])
print(without_mask_files[-5:])

['without_mask_1419.jpg', 'without_mask_714.jpg', 'without_mask_2334.jpg', 'without_mask_1998.jpg', 'without_mask_2733.jpg']
['without_mask_1807.jpg', 'without_mask_644.jpg', 'without_mask_2945.jpg', 'without_mask_1214.jpg', 'without_mask_1604.jpg']


In [14]:
print('number of with mak images',len(with_mask_files))
print('number of without mak images',len(without_mask_files))


number of with mak images 3725
number of without mak images 3828


creating labels for the two class of images

with mask-->1
without mask-->0
